# BT4012 Project: Model Building, Tuning, and Evaluation
### Completed by: Group 8

# Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Load Datasets & Basic EDA

#### Train set

In [2]:
train_df = pd.read_csv('../data/processed/train.csv')
train_df.head()

,merchant_fraud_Abbott-Rogahn,merchant_fraud_Abbott-Steuber,merchant_fraud_Abernathy and Sons,merchant_fraud_Abshire PLC,"merchant_fraud_Adams, Kovacek and Kuhlman",merchant_fraud_Adams-Barrows,"merchant_fraud_Altenwerth, Cartwright and Koss",merchant_fraud_Altenwerth-Kilback,merchant_fraud_Ankunding LLC,merchant_fraud_Ankunding-Carroll,...,merch_lat,merch_long,trans_day,trans_hour,trans_min,trans_month,trans_sec,trans_year,unix_time,is_fraud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.973209,-90.617086,17,23,50,2,4,2020,0.770206,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.887077,-81.564535,9,15,28,8,35,2019,0.412355,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.146606,-81.619032,5,16,3,6,25,2019,0.291471,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,42.358517,-91.763561,6,10,51,11,19,2019,0.577575,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,40.055180,-74.132404,11,8,18,8,23,2019,0.415520,0


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232668 entries, 0 to 232667
Columns: 1675 entries, merchant_fraud_Abbott-Rogahn to is_fraud
dtypes: float64(1661), int64(11), object(3)
memory usage: 2.9+ GB


In [4]:
train_df.describe()

,merchant_fraud_Abbott-Rogahn,merchant_fraud_Abbott-Steuber,merchant_fraud_Abernathy and Sons,merchant_fraud_Abshire PLC,"merchant_fraud_Adams, Kovacek and Kuhlman",merchant_fraud_Adams-Barrows,"merchant_fraud_Altenwerth, Cartwright and Koss",merchant_fraud_Altenwerth-Kilback,merchant_fraud_Ankunding LLC,merchant_fraud_Ankunding-Carroll,...,merch_lat,merch_long,trans_day,trans_hour,trans_min,trans_month,trans_sec,trans_year,unix_time,is_fraud
count,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,...,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000,232668.000000
mean,0.000808,0.001079,0.001036,0.001341,0.000640,0.000692,0.002205,0.001517,0.002600,0.000503,...,38.624009,-90.099380,15.798408,13.439102,29.624504,5.864047,29.650154,2019.294944,0.504212,0.500000
std,0.028414,0.032827,0.032167,0.036595,0.025298,0.026296,0.046904,0.038922,0.050927,0.022419,...,5.173060,14.102614,8.775225,8.397667,17.317171,3.437476,17.288839,0.456019,0.287533,0.500001
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,19.031242,-166.664828,1.000000,0.000000,0.000000,1.000000,0.000000,2019.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,34.941426,-96.845240,8.000000,4.000000,15.000000,3.000000,15.000000,2019.000000,0.258872,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,39.433618,-87.175490,16.000000,15.000000,30.000000,5.000000,30.000000,2019.000000,0.507760,0.500000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,41.987099,-80.018752,23.000000,22.000000,45.000000,9.000000,45.000000,2020.000000,0.740545,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,67.510267,-66.983329,31.000000,23.000000,59.000000,12.000000,59.000000,2020.000000,1.000000,1.000000


#### Validation set

In [5]:
validation_df = pd.read_csv('../data/processed/validation.csv')
validation_df.head()

,merchant_fraud_Abbott-Rogahn,merchant_fraud_Abbott-Steuber,merchant_fraud_Abernathy and Sons,merchant_fraud_Abshire PLC,"merchant_fraud_Adams, Kovacek and Kuhlman",merchant_fraud_Adams-Barrows,"merchant_fraud_Altenwerth, Cartwright and Koss",merchant_fraud_Altenwerth-Kilback,merchant_fraud_Ankunding LLC,merchant_fraud_Ankunding-Carroll,...,merch_lat,merch_long,trans_day,trans_hour,trans_min,trans_month,trans_sec,trans_year,unix_time,is_fraud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.512218,-82.292661,3,14,32,3,35,2020,0.795532,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,44.034668,-85.607445,30,21,32,12,22,2019,0.678867,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32.598802,-96.689835,19,23,13,12,48,2019,0.658533,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.889060,-75.542567,19,14,48,1,11,2020,0.715553,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,43.643013,-70.210856,11,13,45,5,14,2019,0.244782,0


In [6]:
validation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Columns: 1675 entries, merchant_fraud_Abbott-Rogahn to is_fraud
dtypes: float64(1661), int64(11), object(3)
memory usage: 383.4+ MB


In [7]:
validation_df.describe()

,merchant_fraud_Abbott-Rogahn,merchant_fraud_Abbott-Steuber,merchant_fraud_Abernathy and Sons,merchant_fraud_Abshire PLC,"merchant_fraud_Adams, Kovacek and Kuhlman",merchant_fraud_Adams-Barrows,"merchant_fraud_Altenwerth, Cartwright and Koss",merchant_fraud_Altenwerth-Kilback,merchant_fraud_Ankunding LLC,merchant_fraud_Ankunding-Carroll,...,merch_lat,merch_long,trans_day,trans_hour,trans_min,trans_month,trans_sec,trans_year,unix_time,is_fraud
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,0.001267,0.001367,0.001467,0.001700,0.000867,0.001167,0.001467,0.002300,0.001033,0.000500,...,38.579220,-90.303098,15.594233,12.771633,29.626867,6.182000,29.399700,2019.284767,0.514849,0.005800
std,0.035568,0.036944,0.038270,0.041197,0.029427,0.034137,0.038270,0.047904,0.032129,0.022355,...,5.090231,13.813347,8.801771,6.789449,17.394081,3.400391,17.326631,0.451311,0.274809,0.075938
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,19.041880,-166.656560,1.000000,0.000000,0.000000,1.000000,0.000000,2019.000000,0.000022,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,34.768844,-96.976487,8.000000,7.000000,15.000000,3.000000,14.000000,2019.000000,0.294137,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,39.414335,-87.496679,15.000000,14.000000,30.000000,6.000000,29.000000,2019.000000,0.513861,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,41.984748,-80.251130,23.000000,19.000000,45.000000,9.000000,44.000000,2020.000000,0.732459,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,66.554249,-66.962913,31.000000,23.000000,59.000000,12.000000,59.000000,2020.000000,0.999940,1.000000


#### Test set

In [8]:
test_df = pd.read_csv('../data/processed/test.csv')
test_df.head()

,merchant_fraud_Abbott-Rogahn,merchant_fraud_Abbott-Steuber,merchant_fraud_Abernathy and Sons,merchant_fraud_Abshire PLC,"merchant_fraud_Adams, Kovacek and Kuhlman",merchant_fraud_Adams-Barrows,"merchant_fraud_Altenwerth, Cartwright and Koss",merchant_fraud_Altenwerth-Kilback,merchant_fraud_Ankunding LLC,merchant_fraud_Ankunding-Carroll,...,merch_lat,merch_long,trans_day,trans_hour,trans_min,trans_month,trans_sec,trans_year,unix_time,is_fraud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.809923,-118.111582,19,12,30,10,31,2020,1.223274,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.644354,-87.480427,13,5,57,8,47,2020,1.098117,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.784813,-123.198143,26,5,6,8,42,2020,1.122237,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,42.893355,-70.856259,20,18,44,9,30,2020,1.169805,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,33.597864,-83.314567,16,13,22,12,16,2020,1.331246,0


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Columns: 1675 entries, merchant_fraud_Abbott-Rogahn to is_fraud
dtypes: float64(1661), int64(11), object(3)
memory usage: 383.4+ MB


In [10]:
test_df.describe()

,merchant_fraud_Abbott-Rogahn,merchant_fraud_Abbott-Steuber,merchant_fraud_Abernathy and Sons,merchant_fraud_Abshire PLC,"merchant_fraud_Adams, Kovacek and Kuhlman",merchant_fraud_Adams-Barrows,"merchant_fraud_Altenwerth, Cartwright and Koss",merchant_fraud_Altenwerth-Kilback,merchant_fraud_Ankunding LLC,merchant_fraud_Ankunding-Carroll,...,merch_lat,merch_long,trans_day,trans_hour,trans_min,trans_month,trans_sec,trans_year,unix_time,is_fraud
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.0,30000.000000,30000.000000
mean,0.001833,0.001700,0.001367,0.001300,0.000933,0.001767,0.001367,0.001967,0.001533,0.000733,...,38.528262,-90.188371,16.461567,12.814867,29.462767,9.502800,29.360667,2020.0,1.190495,0.003867
std,0.042779,0.041197,0.036944,0.036033,0.030537,0.041995,0.036944,0.044304,0.039128,0.027071,...,5.072750,13.647435,8.929504,6.791867,17.349507,1.974265,17.236350,0.0,0.111907,0.062063
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,19.027422,-166.669812,1.000000,0.000000,0.000000,6.000000,0.000000,2020.0,1.000002,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,34.764436,-96.890949,9.000000,7.000000,14.000000,8.000000,14.000000,2020.0,1.091130,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,39.395308,-87.423744,17.000000,14.000000,29.000000,10.000000,29.000000,2020.0,1.190320,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,41.950251,-80.217881,24.000000,19.000000,45.000000,12.000000,44.000000,2020.0,1.302454,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,66.671540,-66.985234,31.000000,23.000000,59.000000,12.000000,59.000000,2020.0,1.359974,1.000000


#### Splitting into X and y

In [11]:
# split train df, validation df, test df into each of their X and y
y_train = train_df.pop('is_fraud')
X_train = train_df

y_valid = validation_df.pop('is_fraud')
X_valid = validation_df

y_test = test_df.pop('is_fraud')
X_test = test_df

# Model Building & Hyperparameter Tuning

### Steps
- Choose a model: LogisticRegression, RidgeClassifier, SGDClassifier, KNeighborsClassifier, SVC, GaussianNB, DecisionTreeClassifier, HistGradientBoostingClassifier, RandomForestClassifier, VotingClassifier, StackingClassifier, AdaBoostClassifier, XGBClassifier
- Fit the model to the train set
- Get model's predictions for the train and validation set
- Compare the accuracy scores for the validation set and train set to detect overfitting
- Do k-fold cross validation and hyperparameter tuning: GridSearchCV, RandomizedSearchCV
- Repeat the steps above for other models

#### Baseline Model (DummyClassifier)

In [12]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="uniform", random_state=42)
dummy_clf.fit(X_train, y_train)

DummyClassifier(random_state=42, strategy='uniform')

In [13]:
acc_train_dummy = dummy_clf.score(X_train, y_train)
acc_valid_dummy = dummy_clf.score(X_valid, y_valid)
print('Mean accuracy (train):', acc_train_dummy)
print('Mean accuracy (validation):', acc_valid_dummy)
print('Overfitted to train?', acc_train_dummy > acc_valid_dummy)

Mean accuracy (train): 0.4987449928653704
Mean accuracy (validation): 0.4993
Overfitted to train? False


# Model Evaluation

### Steps
- Get a model's predictions for the test set
- From the test set predictions, compute scores for evaluation metrics such as Accuracy, Precision, Recall, F1-Score, ROC AUC
- Repeat the steps above for other models
- Consolidate the evaluation scores for all the models in a table for comparison
- Determine the best performing model for our project’s use case

#### Baseline Model (DummyClassifier)

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# get test set predictions
y_testpred_dummy = dummy_clf.predict(X_test)

# compute scores for evaluation metrics: accuracy, precision, recall, f1-score, roc auc
acc_test_dummy = accuracy_score(y_test, y_testpred_dummy)
prec_test_dummy = precision_score(y_test, y_testpred_dummy)
recall_test_dummy = recall_score(y_test, y_testpred_dummy)
f1_test_dummy = f1_score(y_test, y_testpred_dummy)
auc_test_dummy = roc_auc_score(y_test, y_testpred_dummy)

metric_cols = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC AUC']
dummy_score_df = pd.DataFrame([[acc_test_dummy, prec_test_dummy, recall_test_dummy, f1_test_dummy, auc_test_dummy]], columns=metric_cols, index=['DummyClassifier'])
dummy_score_df

,Accuracy,Precision,Recall,F1-Score,ROC AUC
DummyClassifier,0.499367,0.003796,0.491379,0.007533,0.495388
